In [7]:
def formata_sequencia(sequencia): 
  sequencia = sequencia.upper()
        
  if sequencia[0] == ">":
    sequencia = sequencia.splitlines()
    sequencia = sequencia[1:]
    sequencia = "".join(sequencia).strip()
    sequencia = "*" + sequencia
        
  else:
    sequencia = sequencia.splitlines()
    sequencia = "".join(sequencia).strip()
    sequencia = "*" + sequencia 
        
  return sequencia

In [8]:
def cria_matriz_subs():
  matriz = {"col": ["A", "C", "G", "T"],
          "A": [4, -2, -1, -2],
          "C": [-2, 4, -2, -1],
          "G": [-1, -2, 4, -2],
          "T": [-2, -1, -2, 4]}

  return matriz


def calcula_score(base1, base2, matriz_subs):
  j = matriz_subs["col"].index(base1)
  for base in "ACGT":
    if base2 == base:
      score = matriz_subs[base2][j]
      
      return score
      

'''
Retorna o maior valor do L
'''

def maximo(base1, base2, lado, cima, diagonal):
  # match
  if (base1 == base2) and (diagonal > lado) and (diagonal > cima):
    return diagonal

  # mismatch
  elif (base1 != base2) and (diagonal > lado) and (diagonal > cima):
    return diagonal

  # indel
  elif (lado > cima) and (lado > diagonal):
    return lado
    
  # indel
  else:
    return cima

'''
Retorna o ponteiro do maior valor do L
'''

def ponteiro(base1, base2, lado, cima, diagonal):
  # match
  if (base1 == base2) and (diagonal > lado) and (diagonal > cima):
    return "\\"

  # mismatch
  elif (base1 != base2) and (diagonal > lado) and (diagonal > cima):
    return "\\"

  # indel
  elif (lado > cima) and (lado > diagonal):
    return "-"
    
  # indel
  else:
    return "|"

In [9]:
'''
Montar as matrizes:
- pontuação (preenchida por zeros)
- ponteiros (primeira linha preenchida com '-', primeira coluna preenchida com '|', resto preenchido com '')
'''

def lcs_local(seq1, seq2, matriz_subs):

  pontuacao = []
  ponteiros = []
  g = -3

  # preencher a matriz
  for i in range(0, len(seq1)):
    pontuacao.append([0] * len(seq2))
    ponteiros.append([""] * len(seq2))

  # preencher a primeira linha e a primeira coluna 
  for i in range(0, len(seq1)):
    ponteiros[i][0] = "|"
  for j in range(0, len(seq2)):
    ponteiros[0][j] = "-"

  # linha
  for i in range(1, len(seq1)):
    # coluna
    for j in range(1, len(seq2)):
      # devolver a valor maximo do L

      base1 = seq1[i]
      base2 = seq2[j] 

      s = calcula_score(base1, base2, matriz_subs)
    
      lado = pontuacao[i][j-1]
      cima = pontuacao[i-1][j]
      diagonal = pontuacao[i-1][j-1]
                               
      pontuacao[i][j] = max(0, maximo(base1, base2, lado + g, cima + g, diagonal + s))
      ponteiros[i][j] = ponteiro(base1, base2, lado + g, cima + g, diagonal + s)

  return pontuacao, ponteiros
  

def procura_maior_valor(seq1, pontuacao):
  
  maior_valor_matriz = 0

  for i in range(len(seq1)):
    maior_valor_da_linha = max(pontuacao[i])
    if maior_valor_da_linha > maior_valor_matriz:
      maior_valor_matriz = maior_valor_da_linha
      linha = i
  
  coluna = pontuacao[linha].index(maior_valor_matriz)
  return linha, coluna


'''
Alinha as sequências
'''

def alinhamento_local(seq1, seq2, ponteiros, matriz_subs):
  ali_seq1 = ""
  ali_seq2 = ""
  g = -3
  match = 0
  mismatch = 0
  gap = 0
  score_final = 0

  i, j = procura_maior_valor(seq1, pontuacao)
  valor = pontuacao[i][j]

  while valor > 0:

    s = calcula_score(seq1[i], seq2[j], matriz_subs)    

    if ponteiros[i][j] == "\\" and seq1[i] == seq2[j]:
      ali_seq1 = seq1[i] + ali_seq1
      ali_seq2 = seq2[j] + ali_seq2
      match += 1
      score_final += s
      i -= 1
      j -= 1
      valor = pontuacao[i][j]

    elif ponteiros[i][j] == "\\" and seq1[i] != seq2[j]:
      ali_seq1 = seq1[i] + ali_seq1
      ali_seq2 = seq2[j] + ali_seq2
      mismatch += 1
      score_final += s
      i -= 1
      j -= 1    
      valor = pontuacao[i][j]
    
    elif ponteiros[i][j] == "-":
      ali_seq1 = "-" + ali_seq1
      ali_seq2 = seq2[j] + ali_seq2
      gap += 1
      score_final += g
      j -= 1
      valor = pontuacao[i][j]
    
    elif ponteiros[i][j] == "|":
      ali_seq1 = seq1[i] + ali_seq1
      ali_seq2 = "-" + ali_seq2
      gap += 1
      score_final += g
      i -= 1
      valor = pontuacao[i][j]


  print(f"Matches = {match}")
  print(f"Mismatches = {mismatch}")
  print(f"Gaps = {gap}")
  print(f"Score final = {score_final}")
  print(ali_seq1)  
  print(ali_seq2)  

In [10]:
'''
Imprimir a matriz
'''

def imprime_matriz(seq1, seq2, pontuacao, ponteiros):
  print("\t", end="")

  for j in range(0, len(seq2)):
    print(seq2[j], end="\t")
  print()
  for i in range(0, len(seq1)):
    print(seq1[i], end="\t")

    for j in range(0, len(seq2)):
      print(pontuacao[i][j], ponteiros[i][j], end="\t", sep="")
    print()
  print()



In [11]:
seq1 = "ATGCAATGGTCCTAAGG"
seq2 = "TGCATGG"
seq1 = formata_sequencia(seq1)
seq2 = formata_sequencia(seq2)

matriz_subs = cria_matriz_subs()
pontuacao, ponteiros = lcs_local(seq1, seq2, matriz_subs)
alinhamento_local(seq1, seq2, ponteiros, matriz_subs)



Matches = 7
Mismatches = 0
Gaps = 1
Score final = 25
TGCAATGG
TGCA-TGG


In [12]:
imprime_matriz(seq1, seq2, pontuacao, ponteiros)

	*	T	G	C	A	T	G	G	
*	0-	0-	0-	0-	0-	0-	0-	0-	
A	0|	0\	0\	0\	4\	1-	0\	0\	
T	0|	4\	1-	0\	1|	8\	5-	2-	
G	0|	1|	8\	5-	2-	5|	12\	0|	
C	0|	0\	5|	12\	9-	6-	9|	10\	
A	0|	0\	2|	9|	16\	13-	10-	8\	
A	0|	0\	0|	6|	13|	14\	12\	5|	
T	0|	4\	1-	3|	10|	17\	14-	11-	
G	0|	1|	8\	5-	7|	14|	21\	8|	
G	0|	0|	5|	6\	4|	11|	18|	25\	
T	0|	4\	2|	4\	4\	8|	15|	22|	
C	0|	1|	2\	6\	3-	5|	12|	19|	
C	0|	0\	0|	6\	4\	2|	9|	16|	
T	0|	4\	1-	3|	4\	8\	6|	13|	
A	0|	1|	3\	0|	7\	5|	7\	10|	
A	0|	0|	0|	1\	4|	5\	4|	7|	
G	0|	0\	4\	1-	1|	2|	9\	8\	
G	0|	0\	4\	2\	0\	0|	6|	13\	

